In [1]:
import os
import msal
from msal.application import ConfidentialClientApplication
from dotenv import load_dotenv
import jwt
import json
import requests
from datetime import datetime, timedelta
from typing import Optional, Dict, List
from datetime import datetime, timedelta, timezone

# Define the target directory
target_directory = r'C:\Users\pablosal\Desktop\sharepoint-indexing-azure-cognitive-search'

# Load .env file
load_dotenv()

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

#load logging 
from utils.ml_logging import get_logger
logger = get_logger()

Directory changed to C:\Users\pablosal\Desktop\sharepoint-indexing-azure-cognitive-search


In [ ]:
# There needs to be a config.json file that looks like this:
    
# {
#     "authority": "https://login.microsoftonline.com/XXX",
#     "client_id": "XXX",
#     "scope": [ "https://graph.microsoft.com/.default" ],
#     "secret": "XXX",
#     "site_domain": "XXX.sharepoint.com",
#     "site_name": "Mark8ProjectTeam",
#     "include_auth_info": true,
#     "search_service_name": "XXX",
#     "search_index_name": "sharepoint-index-1",
#     "search_admin_api_key": "XXX"
# }

In [2]:
accessToken = None 
requestHeaders = None 
tokenExpiry = None 
queryResults = None 

In [3]:
# Get the variables
tenantID = os.getenv('TENANT_ID')
clientID = os.getenv('CLIENT_ID')
clientSecret = os.getenv('CLIENT_SECRET')
graphURI = 'https://graph.microsoft.com'
authority = 'https://login.microsoftonline.com/' + tenantID
scope = ['https://graph.microsoft.com/.default']

In [4]:
app = msal.ConfidentialClientApplication(
    client_id=clientID,
    authority=f'https://login.microsoftonline.com/{tenantID}',
    client_credential=clientSecret
)

In [5]:
# Acquire a token for the Graph API
result = app.acquire_token_silent(scope, account=None)
if not result:
    result = app.acquire_token_for_client(scopes=scope)

# Check if the token was acquired
if 'access_token' in result:
    token = result['access_token']
else:
    print(result.get('error'))
    print(result.get('error_description'))
    print(result.get('correlation_id'))
    exit()

In [6]:
# Call the Graph API
graph_url = 'https://graph.microsoft.com/v1.0'

In [7]:
# Get the current user
user = requests.get(
    f'{graph_url}/me',
    headers={'Authorization': 'Bearer ' + token}
).json()
print(user)

# Get the groups the user belongs to
groups = requests.get(
    f'{graph_url}/me/memberOf',
    headers={'Authorization': 'Bearer ' + token}
).json()

{'error': {'code': 'BadRequest', 'message': '/me request is only valid with delegated authentication flow.', 'innerError': {'date': '2023-12-09T01:09:02', 'request-id': '4eda5fed-6101-472e-8250-f44d34d433f5', 'client-request-id': '4eda5fed-6101-472e-8250-f44d34d433f5'}}}


In [53]:
# Replace these with your values
tenantID = os.getenv('TENANT_ID')
clientID = os.getenv('CLIENT_ID')
username = 'admin@MngEnvMCAP747548.onmicrosoft.com'
password = 'your_password'
scope = ['https://graph.microsoft.com/.default']

In [ ]:
# Create a public client application
app = msal.PublicClientApplication(
    client_id=client_id,
    authority=f'https://login.microsoftonline.com/{tenant_id}'
)

# Acquire a token for the Graph API
result = app.acquire_token_by_username_password(
    username=username,
    password=password,
    scopes=scope
)

In [ ]:
# Create a public client application
app = msal.PublicClientApplication(
    client_id=client_id,
    authority=f'https://login.microsoftonline.com/{tenant_id}'
)

# Acquire a token for the Graph API
result = app.acquire_token_by_username_password(
    username=username,
    password=password,
    scopes=scope
)

# Check if the token was acquired
if 'access_token' in result:
    token = result['access_token']
else:
    print(result.get('error'))
    print(result.get('error_description'))
    print(result.get('correlation_id'))
    exit()

# Call the Graph API
graph_url = 'https://graph.microsoft.com/v1.0'

In [ ]:
# Get the current user
user = requests.get(
    f'{graph_url}/me',
    headers={'Authorization': 'Bearer ' + token}
).json()
print(user)

# Get the groups the user belongs to
groups = requests.get(
    f'{graph_url}/me/memberOf',
    headers={'Authorization': 'Bearer ' + token}
).json()
print(groups)

In [25]:
from msal import PublicClientApplication

app = PublicClientApplication(tenantID, authority="https://login.microsoftonline.com/" + tenantID)
accounts = app.get_accounts()

In [28]:
result = app.acquire_token_silent(scope,account=None)

In [29]:
import requests
headers = {'Authorization': 'Bearer ' + result['access_token']}
data = {'securityEnabledOnly': False}
response = requests.post('https://graph.microsoft.com/v1.0/me/getMemberGroups', headers=headers, json=data)
if response.ok:
    groups = response.json()['value']
    print(groups) # This will print the lis

TypeError: 'NoneType' object is not subscriptable

In [61]:
def msgraph_auth(client_id: str, client_secret: str, tenant_id: str, authority: Optional[str] = None, scope: Optional[List[str]] = None) -> ConfidentialClientApplication:
    """
    Authenticate with Microsoft Graph using MSAL for Python.

    This function authenticates with Microsoft Graph using the MSAL library. It attempts to acquire a token silently,
    and if that fails, it tries to acquire a new token. It decodes and prints the acquired access token and its expiry time.

    :param client_id: The application (client) ID of your Azure AD app registration.
    :param client_secret: The client secret for your Azure AD app registration.
    :param tenant_id: The tenant ID for your Azure AD tenant.
    :param authority: The authority URL for your Azure AD tenant. Optional, defaults to 'https://login.microsoftonline.com/{tenant_id}'.
    :param scope: Scopes required for the token. Optional, defaults to ['https://graph.microsoft.com/.default'].
    :return: A dictionary with the access token and its expiry time if successful, None otherwise.
    :raises Exception: If there are issues in acquiring or decoding the token.
    """
    if authority is None:
        authority = f'https://login.microsoftonline.com/{tenant_id}'

    if scope is None:
        scope = ['https://graph.microsoft.com/.default']

    app = msal.ConfidentialClientApplication(client_id, authority=authority, client_credential=client_secret)
    
    try:
        access_token = app.acquire_token_silent(scope, account=None)
        if not access_token:
            access_token = app.acquire_token_for_client(scopes=scope)
            if 'access_token' in access_token:
                logger.info('New access token retrieved....')
            else:
                logger.error('Error acquiring authorization token. Check your tenantID, clientID, and clientSecret.')
                return None
        else:
            logger.info('Token retrieved from MSAL Cache....')

        algorithms = ["RS256"]
        decoded_access_token = jwt.decode(access_token['access_token'], algorithms=algorithms, options={"verify_signature": False})
        access_token_formatted = json.dumps(decoded_access_token, indent=2)
        logger.info('Decoded Access Token:\n%s', access_token_formatted)

        # Token Expiry
        token_expiry = datetime.fromtimestamp(int(decoded_access_token['exp']))
        logger.info('Token Expires at: %s', str(token_expiry))
        return access_token
    except Exception as err:
        logger.error("Error in msgraph_auth: %s", str(err))
        raise

In [62]:
from docx import Document
import io

In [63]:
def make_ms_graph_request(access_token: str, url: str) -> Dict:
    """
    Make a request to the Microsoft Graph API.

    :param access_token: The access token for Microsoft Graph API authentication.
    :param url: The URL for the Microsoft Graph API endpoint.
    :return: The JSON response from the Microsoft Graph API.
    :raises Exception: If there's an HTTP error or other issues in making the request.
    """
    headers = {"Authorization": f"Bearer {access_token}"}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as err:
        logger.error(f"HTTP Error: {err}")
        raise
    except Exception as err:
        logger.error(f"Error in make_ms_graph_request: {err}")
        raise

def get_drive_id(access_token: str, site_id: str) -> str:
    """
    Get the drive ID from a Microsoft Graph site.

    :param access_token: The access token for Microsoft Graph API authentication.
    :param site_id: The site ID in Microsoft Graph.
    :return: The drive ID.
    :raises Exception: If there's an error in fetching the drive ID.
    """
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive"
    try:
        json_response = make_ms_graph_request(access_token, url)
        return json_response["id"]
    except Exception as err:
        logger.error(f"Error in get_drive_id: {err}")
        raise

def get_files_in_site(access_token: str, site_id: str, drive_id: str, minutes_ago: Optional[int] = None, file_formats: Optional[List[str]] = None) -> List[Dict]:
    """
    Get a list of files in a site's drive, optionally filtered by creation or last modification time and file formats.

    :param access_token: The access token for Microsoft Graph API authentication.
    :param site_id: The site ID in Microsoft Graph.
    :param drive_id: The drive ID in Microsoft Graph.
    :param minutes_ago: Optional integer to filter files created or updated within the specified number of minutes from now. If None, no time-based filtering is applied.
    :param file_formats: List of desired file formats.
    :return: A list of file details.
    :raises Exception: If there's an error in fetching file details.
    """
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drives/{drive_id}/root/children"
    try:
        logger.info('Making request to Microsoft Graph API')
        json_response = make_ms_graph_request(access_token, url)
        files = json_response["value"]
        logger.info('Received response from Microsoft Graph API')   

        time_limit = datetime.now(timezone.utc) - timedelta(minutes=minutes_ago) if minutes_ago is not None else None

        filtered_files = [file for file in files if (
            (time_limit is None or 
             datetime.fromisoformat(file['fileSystemInfo']['createdDateTime'].rstrip('Z')).replace(tzinfo=timezone.utc) >= time_limit or 
             datetime.fromisoformat(file['fileSystemInfo']['lastModifiedDateTime'].rstrip('Z')).replace(tzinfo=timezone.utc) >= time_limit) and
            (not file_formats or any(file['name'].endswith(f'.{fmt}') for fmt in file_formats))
        )]

        return filtered_files
    except Exception as err:
        logger.error(f"Error in get_files_in_site: {err}")
        raise
 

def get_file_permissions(access_token: str, site_id: str, item_id: str) -> List[Dict]:
    """
    Get the permissions of a file in a site.

    :param access_token: The access token for Microsoft Graph API authentication.
    :param site_id: The site ID in Microsoft Graph.
    :param item_id: The item ID of the file in Microsoft Graph.
    :return: A list of permission details.
    :raises Exception: If there's an error in fetching permission details.
    """
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/items/{item_id}/permissions"

    try:
        json_response = make_ms_graph_request(access_token, url)
        return json_response["value"]
    except Exception as err:
        logger.error(f"Error in get_file_permissions: {err}")
        raise

def group_users_by_role(user_data: List[Dict]) -> str:
    """
    Group users by their roles and return a JSON string representing this grouping.

    This function takes a list of user data dictionaries, each containing user roles, and other attributes.
    It returns a JSON string where the keys are roles and the values are lists of user display names corresponding to each role.

    :param user_data: List of dictionaries containing user data.
    :return: JSON string representing users grouped by their roles.
    :raises DataError: If there are issues with data processing.
    """
    try:
        grouped_users = {}
        for user in user_data:
            roles = user.get('roles', [])
            display_name = user.get('grantedTo', {}).get('user', {}).get('displayName') or \
                           user.get('grantedToV2', {}).get('siteGroup', {}).get('displayName') or \
                           user.get('grantedToV2', {}).get('group', {}).get('displayName')

            for role in roles:
                if role in grouped_users:
                    grouped_users[role].append(display_name)
                else:
                    grouped_users[role] = [display_name]

        return json.dumps(grouped_users)
    except Exception as e:
        logger.error(f"Error processing user data: {e}")

def get_docx_content(site_id: str, file_name: str, access_token: str) -> Optional[str]:
    """
    Retrieve the text content from a .docx file in a specific site drive.

    :param site_id: The site ID in Microsoft Graph.
    :param file_name: The name of the .docx file.
    :param access_token: The access token for Microsoft Graph API authentication.
    :return: Text content of the docx file or None if there's an error.
    """
    endpoint = f'https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root:/{file_name}'
    try:
        logger.info(f'Making request for file content: {file_name}')
        file_content_response = make_ms_graph_request(endpoint, access_token)
        file_content = file_content_response.content

        document = Document(io.BytesIO(file_content))
        content = "\n".join([paragraph.text for paragraph in document.paragraphs])
        return content
    except Exception as err:
        logger.error(f"Error retrieving content for {file_name}: {err}")
        return None
    
def get_docx_content(site_id: str, file_name: str, drive_id: str, access_token: str) -> Optional[str]:
    """
    Retrieve the text content from a .docx file in a specific site drive.

    :param site_id: The site ID in Microsoft Graph.
    :param file_name: The name of the .docx file.
    :param drive_id: The drive ID in Microsoft Graph.
    :param access_token: The access token for Microsoft Graph API authentication.
    :return: Text content of the docx file or None if there's an error.
    """
    
    endpoint = f'https://graph.microsoft.com/v1.0/sites/{site_id}/drives/{drive_id}/root:/{file_name}:/content'
    try:
        logger.info(f'Starting request for file: {file_name}')
        response = requests.get(endpoint, headers={'Authorization': 'Bearer ' + access_token})

        # Log different outcomes of the response
        if response.status_code != 200:
            logger.error(f"Failed to retrieve file content. Status code: {response.status_code}, Response: {response.text}")
            return None

        file_content = response.content
        document = Document(io.BytesIO(file_content))
        content = "\n".join([paragraph.text for paragraph in document.paragraphs])

        logger.info(f'Successfully retrieved content for file: {file_name}')
        return content
    except requests.exceptions.RequestException as req_err:
        logger.error(f"Request error retrieving content for {file_name}: {req_err}")
        return None
    except Exception as err:
        logger.error(f"General error retrieving content for {file_name}: {err}")
        return None

In [64]:
client_auth = msgraph_auth(clientID,clientSecret,tenantID)

2023-12-08 18:44:05,171 - micro - MainProcess - INFO     New access token retrieved.... (1613506430.py:msgraph_auth:29)
2023-12-08 18:44:05,178 - micro - MainProcess - INFO     Decoded Access Token:
{
  "aud": "https://graph.microsoft.com",
  "iss": "https://sts.windows.net/9495d8c9-4ebb-4107-b905-c7b45d1b7b7a/",
  "iat": 1702082345,
  "nbf": 1702082345,
  "exp": 1702086245,
  "aio": "E2VgYNj8br7t7bUmT1h/Ha+ZnzS3FgA=",
  "app_displayname": "dev-graph",
  "appid": "118583ee-94ed-45dd-870b-73784045eb37",
  "appidacr": "1",
  "idp": "https://sts.windows.net/9495d8c9-4ebb-4107-b905-c7b45d1b7b7a/",
  "idtyp": "app",
  "oid": "4f614374-65fa-45fc-8369-cb616a6fe08f",
  "rh": "0.Ab0AydiVlLtOB0G5Bce0XRt7egMAAAAAAAAAwAAAAAAAAADLAAA.",
  "roles": [
    "TeamsActivity.Read.All",
    "SharePointTenantSettings.Read.All",
    "People.Read.All",
    "Sites.Read.All",
    "Sites.Manage.All",
    "Directory.Read.All",
    "OnlineMeetingTranscript.Read.All",
    "BrowserSiteLists.ReadWrite.All",
    "File

In [14]:
site_domain = 'mngenvmcap747548.sharepoint.com'
site_name = 'Contoso'

In [15]:
print ('Getting the Site ID...')
endpoint = 'https://graph.microsoft.com/v1.0/sites/' + f"{site_domain}" + ':/sites/' +  f"{site_name}" + ':/'
result = make_ms_graph_request(client_auth['access_token'], endpoint)

Getting the Site ID...


In [16]:
print ('Getting the Site ID...')
endpoint = 'https://graph.microsoft.com/v1.0/sites/' + f"{site_domain}" + ':/sites/' +  f"{site_name}" + ':/'
result = make_ms_graph_request(client_auth['access_token'], endpoint)
# site_id = result.json()["id"].split(',')[1]
site_id = result["id"]
print ('Site ID:', site_id)

Getting the Site ID...
Site ID: mngenvmcap747548.sharepoint.com,877fe60f-a62d-4ed1-8eda-af543c437d2c,ac47d8a7-cd54-4344-bd9d-26ada5a075c0


In [17]:
print ('Getting the Drive ID...')
drive_id = get_drive_id(client_auth['access_token'], site_id)
drive_id

Getting the Drive ID...


'b!D-Z_hy2m0U6O2q9UPEN9LKfYR6xUzURDvZ0mraWgdcAot0GWx37EQLiVD3sO7-vm'

In [18]:
files = get_files_in_site(client_auth['access_token'], site_id=site_id,drive_id=drive_id,minutes_ago=None,file_formats=['docx'])

2023-12-08 11:59:12,731 - micro - MainProcess - INFO     Making request to Microsoft Graph API (2387473689.py:get_files_in_site:53)
2023-12-08 11:59:13,304 - micro - MainProcess - INFO     Received response from Microsoft Graph API (2387473689.py:get_files_in_site:56)


In [19]:
test = get_file_permissions(client_auth['access_token'], site_id=site_id, item_id='01W3WT6PHBIOXRSZRV2JEZN4VVOFZ4M3Q7')

In [20]:
test

[{'id': 'Q29udG9zbyBPd25lcnM',
  'roles': ['owner'],
  'shareId': 'Q29udG9zbyBPd25lcnM',
  'grantedToV2': {'siteGroup': {'displayName': 'Contoso Owners',
    'id': '3',
    'loginName': 'Contoso Owners'}},
  'grantedTo': {'user': {'displayName': 'Contoso Owners'}},
  'inheritedFrom': {}},
 {'id': 'Q29udG9zbyBWaXNpdG9ycw',
  'roles': ['read'],
  'shareId': 'Q29udG9zbyBWaXNpdG9ycw',
  'grantedToV2': {'siteGroup': {'displayName': 'Contoso Visitors',
    'id': '4',
    'loginName': 'Contoso Visitors'}},
  'grantedTo': {'user': {'displayName': 'Contoso Visitors'}},
  'inheritedFrom': {}},
 {'id': 'Q29udG9zbyBNZW1iZXJz',
  'roles': ['write'],
  'shareId': 'Q29udG9zbyBNZW1iZXJz',
  'grantedToV2': {'siteGroup': {'displayName': 'Contoso Members',
    'id': '5',
    'loginName': 'Contoso Members'}},
  'grantedTo': {'user': {'displayName': 'Contoso Members'}},
  'inheritedFrom': {}},
 {'id': 'Yzowby5jfGZlZGVyYXRlZGRpcmVjdG9yeWNsYWltcHJvdmlkZXJ8NThiNWRlYTYtMWI5ZS00ZjU3LWJiZjYtNjRkOWViNjZmZjExX28',

In [21]:
group_users_by_role(test)

'{"owner": ["Contoso Owners", "Contoso Owners"], "read": ["Contoso Visitors"], "write": ["Contoso Members"]}'

In [74]:
def get_docx_content(site_id: str, file_name: str, drive_id: str, access_token: str) -> Optional[str]:
    """
    Retrieve the text content from a .docx file in a specific site drive.

    :param site_id: The site ID in Microsoft Graph.
    :param file_name: The name of the .docx file.
    :param drive_id: The drive ID in Microsoft Graph.
    :param access_token: The access token for Microsoft Graph API authentication.
    :return: Text content of the docx file or None if there's an error.
    """
    
    endpoint = f'https://graph.microsoft.com/v1.0/sites/{site_id}/drives/{drive_id}/root:/{file_name}:/content'
    try:
        logger.info(f'Starting request for file: {file_name}')
        response = requests.get(endpoint, headers={'Authorization': 'Bearer ' + access_token})

        # Log different outcomes of the response
        if response.status_code != 200:
            logger.error(f"Failed to retrieve file content. Status code: {response.status_code}, Response: {response.text}")
            return None

        file_content = response.content
        document = Document(io.BytesIO(file_content))
        content = "\n".join([paragraph.text for paragraph in document.paragraphs])

        logger.info(f'Successfully retrieved content for file: {file_name}')
        return content
    except requests.exceptions.RequestException as req_err:
        logger.error(f"Request error retrieving content for {file_name}: {req_err}")
        return None
    except Exception as err:
        logger.error(f"General error retrieving content for {file_name}: {err}")
        return None